##SETUP

In [351]:
import os
import time
import pandas as pd
import numpy as np
# Android ADB
from scrcpy import Client, const

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)

bot = bot_core.Bot()
# Evaluate grid to update OCR images
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
display(grid_df)

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,hunter.png,197.0,4,287.0,"[0, 0]",0.0
1,icon_1,hunter.png,174.0,3,120.0,"[0, 1]",0.0
2,icon_2,empty.png,700.0,0,0.0,"[0, 2]",0.0
3,icon_3,zealot.png,365.0,3,172.5,"[0, 3]",0.0
4,icon_4,crystal.png,219.0,1,1744.0,"[0, 4]",0.0
5,icon_5,hunter.png,244.0,2,335.0,"[1, 0]",0.0
6,icon_6,zealot.png,363.0,1,1103.0,"[1, 1]",0.0
7,icon_7,crystal.png,207.0,2,345.0,"[1, 2]",0.0
8,icon_8,hunter.png,191.0,2,270.0,"[1, 3]",0.0
9,icon_9,chemist.png,195.0,3,130.5,"[1, 4]",0.0


In [13]:
# Check if alive
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

In [187]:
bot.getScreen()

In [39]:
# Restart game
bot.restart_RR()

## Watch ads

In [186]:
avail_buttons = bot.get_current_icons(available=True)
pos = bot_core.get_button_pos(avail_buttons,'refresh_button.png')


array([ 81, 306], dtype=int64)

In [200]:
avail_buttons = bot.get_current_icons(available=True)
display(avail_buttons)

,icon,available,"pos [X,Y]"
0,battle_icon.png,True,"(359, 1419)"
1,pvp_button.png,True,"(140, 1259)"


## Get grid meta

In [4]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,crystal.png,185.0,2,605.0,"[0, 0]",1.0
1,icon_1,crystal.png,385.0,3,178.5,"[0, 1]",1.0
2,icon_2,dryad.png,291.0,2,585.0,"[0, 2]",1.0
3,icon_3,monkey.png,556.0,2,610.0,"[0, 3]",1.0
4,icon_4,hunter.png,249.0,3,213.0,"[0, 4]",1.0
5,icon_5,monkey.png,324.0,1,1217.0,"[1, 0]",1.0
6,icon_6,dryad.png,664.0,2,275.0,"[1, 1]",1.0
7,icon_7,empty.png,999.0,0,0.0,"[1, 2]",1.0
8,icon_8,hunter.png,670.0,1,1489.0,"[1, 3]",1.0
9,icon_9,crystal.png,270.0,2,605.0,"[1, 4]",1.0


In [278]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
display(unit_series)
#merge_series = unit_series[unit_series>=2]


unit         rank
empty.png    0       8
crystal.png  2       2
chemist.png  2       1
             3       1
hunter.png   3       1
             5       1
zealot.png   4       1
Name: unit, dtype: int64

In [250]:
#('empty.png',0) in group_keys
df_groups['empty.png']<=2

False

In [252]:
merge_series = unit_series

bot_core.adv_filter_keys(merge_series,[['chemist.png','monkey.png']])

unit         rank
chemist.png  3       1
monkey.png   4       1
Name: unit, dtype: int64

In [437]:
# Setup test
multi_index=pd.MultiIndex.from_tuples([
            ( 'monkey.png', 1),
            ( 'monkey.png', 4),
            ('crystal.png', 2),
            ('crystal.png', 4),
            ('zealot.png', 2),
            ('zealot.png', 4),],
           names=['unit', 'rank'])
unit_series_test=pd.Series([2,2,2,2,2,2],index=multi_index)
# Pre-condition
merge_series = unit_series_test.copy()
merge_series = bot_core.adv_filter_keys(merge_series,'empty.png',remove=True)
merge_chemist = bot_core.adv_filter_keys(unit_series,'chemist.png',remove=False)
if not merge_chemist.empty:
    max_chemist = merge_chemist.index.max()
    # Remove 1 count of highest rank chemist
    merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
## Select stuff to merge
merge_series = merge_series[merge_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
# Remain
#merge_series = bot_core.adv_filter_keys(merge_series,[[3,4,5],'crystal.png'],remove=True)
adv_filter_keys(merge_series,[[4,5],['zealot.png','crystal.png']],remove=True)

#bot_core.adv_filter_keys(merge_series,[['zealot.png','crystal.png']],remove=True)


[4, 5] 4
[4, 5] 5
123
['zealot.png', 'crystal.png'] zealot.png
['zealot.png', 'crystal.png'] crystal.png
123


unit         rank
monkey.png   1       2
             4       2
crystal.png  2       2
zealot.png   2       2
dtype: int64

In [427]:
# Returns all elements which match tokens value with multiple levels
# Either provide nested list of list, simple list or unit type/unit rank and if remove or not
# Criteria example:  [[3,4,5],['zealot.png','crystal.png']]
def adv_filter_keys(unit_series,tokens,remove=False):
    if unit_series.empty:
        return pd.Series(dtype=object)
    if not isinstance(tokens, list): # Make token a list if not already
        tokens = [tokens]
    # Add detection of dimension in input tokens
    merge_series= unit_series.copy()
    for level in tokens:
        merge_series_temp= merge_series.copy()
        if not isinstance(level, list): # Make token a list if not already
            level = [level]
        series= []
        for token in level:
            print(level,token)
            # check if given token is int, assume unit rank filter
            if isinstance(token,int):
                exists = merge_series.index.get_level_values('rank').isin([token]).any()
                if exists:
                    series.append(merge_series.xs(token, level='rank',drop_level=False))
                else: continue # skip if nothing matches criteria
            elif isinstance(token,str):
                if token in merge_series: 
                    series.append(merge_series.xs(token, level='unit',drop_level=False))
                else: continue
        # Every iteration
        # If any matches are found
        if not len(series)==0:
            merge_series = pd.concat(series)
            # Select matches in previous matches 
            merge_series = merge_series_temp[merge_series_temp.index.isin(merge_series.index)]
         # return empty list if empty and nothing matches criteria
        elif not remove:
            return pd.Series(dtype=object)
        # if removing matches from initial series and no matches are found, do nothing this loop, keep list same
        else: 
            continue
    # LOOP DONE
    if remove:
        # Remove all matches found from original series
        merge_series = unit_series[~unit_series.index.isin(merge_series.index)]
    # Return matches found
    return merge_series



        # Otherwise Do next loop with unchanged merge series otherwise
    # Return result of all criterias
    print(555)
    return merge_series
#unit_series

In [331]:

merge_series = unit_series.copy()
merge_series = adv_filter_keys(merge_series,'empty.png',remove=True)
merge_chemist = adv_filter_keys(unit_series,'chemist.png',remove=False)
if not merge_chemist.empty:
    max_chemist = merge_chemist.index.max()
    # Remove 1 count of highest rank chemist
    merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
    display(merge_series)
## Select stuff to merge
merge_series = merge_series[merge_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
merge_prio = adv_filter_keys(merge_series,[['chemist.png','monkey.png']],remove=True)
#if not merge_prio.empty:
#    print('merge!')
display(merge_prio)

unit         rank
crystal.png  2       2
chemist.png  2       1
             3       0
hunter.png   3       1
             5       1
zealot.png   4       1
Name: unit, dtype: int64

unit         rank
crystal.png  2       2
Name: unit, dtype: int64

In [311]:
merge_series

unit         rank
crystal.png  2       2
Name: unit, dtype: int64

In [287]:
unit_series

unit         rank
empty.png    0       8
crystal.png  2       2
chemist.png  2       1
             3       1
hunter.png   3       1
             5       1
zealot.png   4       1
Name: unit, dtype: int64

In [205]:
merge_series = unit_series_test
adv_filter_keys(merge_series,[[3,4,5],['zealot.png','crystal.png']],remove=True)


unit         rank
monkey.png   1       2
crystal.png  2       2
             4       2
zealot.png   2       2
             4       2
dtype: int64

In [ ]:
bot_core.adv_filter_keys(merge_series,[
    [3,4,5],
    ['crystal.png','zealot.png']
    ],remove=True)

In [97]:
# Find highest chemist rank
merge_series = adv_filter_keys(unit_series,'chemist.png',remove=False)
max_chemist = merge_series.index.max()
# Remove 1 count of highest rank chemist
merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
merge_series

unit         rank
chemist.png  4       0
Name: unit, dtype: int64

In [100]:
merge_series = unit_series
merge_prio = adv_filter_keys(merge_series,[['chemist.png','monkey.png']])
if not merge_prio.empty:
    merge_df = self.merge_unit(df_split,merge_prio)

adv_filter_keys(merge_series,[[3,4,5],['zealot.png','crystal.png']],remove=True)


unit         rank
empty.png    0       3
hunter.png   1       2
monkey.png   1       2
chemist.png  4       1
crystal.png  2       1
monkey.png   2       1
             4       1
zealot.png   2       1
Name: unit, dtype: int64

In [34]:
# Remove tuple tokens from unit series
def remove_keys(unit_series,tokens = [('empty.png', 0)]):
    return unit_series[~unit_series.index.isin([tokens])]
merge_series 

array([False,  True, False,  True, False, False, False, False, False,
       False, False])

In [37]:
merge_series[~merge_series.index.isin([('crystal.png', 1),('crystal.png', 2)])]

unit         rank
dryad.png    2       3
empty.png    0       2
crystal.png  3       1
dryad.png    1       1
hunter.png   1       1
             3       1
monkey.png   1       1
             2       1
             3       1
Name: unit, dtype: int64